## 허프 변환에 의한 직선 및 원 검출

Sobel, Log, Canny로 검출한 엣지는 단순히 화소들의 집합입니다. 즉, 엣지는 직선, 사각형, 원 등의 구조적 정보를 갖지 않습니다. 허프 변환을 사용하면 에지에서 직선 또는 원의 방정식의 파라미터를 검출할 수 있습니다. 

허프변환은 이미지에서 모양을 찾는 가장 유명한 방법입니다. 이 방법을 이용하면 이미지의 형태를 찾거나, 누락되거나 깨진 영역을 복원할 수 있습니다.

허프변환 시에는 직선의 방정식을 이용합니다. 하나의 점을 지나는 무수한 직선의 방적식은 y=mx+c로 표현할 수 있으며, 이것을 삼각함수를 이용하여 변형하면 r = 𝑥 cos 𝜃 + 𝑦 sin 𝜃 으로 표현할 수 있습니다.

### cv2.HoughLines(image, rho, theta, threshold) -> lines

- image : 8비트 단일 채널 이미지입니다. 
- rho : r 값의 범위입니다. 0-1 실수입니다.
- theta : 세타 값의 범위입니다. 0-180 정수입니다.
- threshold : 임계값입니다. 작으면 많은 선이 검출되는 만큼 정확도가 떨어지고, 숫자가 크면 정확도가 올라갑니다.

### cv2.HoughLinesP(image, rho, theta, threshold, minLineLength, maxLineGap) -> lines

- image : 8비트 단일 채널 이미지입니다. 
- rho : r 값의 범위입니다. 0-1 실수입니다.
- theta : 세타 값의 범위입니다. 0-180 정수입니다.
- threshold : 임계값입니다. 작으면 많은 선이 검출되는 만큼 정확도가 떨어지고, 숫자가 크면 정확도가 올라갑니다.
- minLineLength : 선의 최소 길이입니다. 이 값보다 작으면 선이 무시됩니다.
- maxLineGap : 선과 선 사이의 최대 허용간격입니다. 이보다 작으면 선이 무시됩니다. 

### cv2.HoughCircles(image, method, dp, minDist, param1, param2, minRadius, maxRadius) -> circles

- image : 8비트 단일 채널 이미지입니다. 
- method : 검출 방법. HOUGH_GRADIENT를 사용합니다.
- dp : dp가 1이면 입력 이미지와 동일한 해상도를 보입니다.
- minDist : 검출한 원의 중심과의 최소거리. 값이 작으면 원이 아닌 것들도 검출이 되고, 너무 크면 원을 놓칠 수 있습니다.
- param1 : 내부적으로 사용하는 캐니엣지 검출기에 전달되는 파라미터입니다.
- param2 : 이 값이 작을 수록 오류가 높고, 크면 검출률이 높아집니다.
- minRadius : 원의 최소 반지름입니다.
- maxRadius : 원의 최대 반지름입니다.

In [3]:
# 허프
import cv2
import numpy as np

src = cv2.imread('./data/rect.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 100)
lines = cv2.HoughLines(edges, rho=1, theta=np.pi/180.0, threshold=100)
print('lines.shape=', lines.shape)

for line in lines:
    rho, theta   = line[0]
    c = np.cos(theta)
    s = np.sin(theta)
    x0 = c*rho
    y0 = s*rho
    x1 = int(x0 + 1000*(-s))
    y1 = int(y0 + 1000*(c))
    x2 = int(x0 - 1000*(-s))
    y2 = int(y0 - 1000*(c))
    cv2.line(src, (x1,y1), (x2,y2), (0,0,255), 2)
    
cv2.imshow('edges',  edges)
cv2.imshow('src',  src)
cv2.waitKey()
cv2.destroyAllWindows()


lines.shape= (4, 1, 2)


In [4]:
# 확률 허프
import cv2
import numpy as np

src = cv2.imread('./data/rect.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 100)
lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180.0, threshold=100)
print('lines.shape=', lines.shape)

for line in lines:
    x1, y1, x2, y2   = line[0]
    cv2.line(src,(x1,y1),(x2,y2),(0,0,255),2)
    
cv2.imshow('edges',  edges)
cv2.imshow('src',  src)
cv2.waitKey()
cv2.destroyAllWindows()


lines.shape= (4, 1, 4)


In [5]:
# 허프 원 검출
import cv2
import numpy as np

#1
src1 = cv2.imread('./data/circles.jpg')
gray1 = cv2.cvtColor(src1,cv2.COLOR_BGR2GRAY)
circles1 = cv2.HoughCircles(gray1, method = cv2.HOUGH_GRADIENT,
            dp=1, minDist=50, param2=15)

circles1 =  np.int32(circles1)
print('circles1.shape=', circles1.shape)
for circle in circles1[0,:]:    
    cx, cy, r  = circle
    cv2.circle(src1, (cx, cy), r, (0,0,255), 2)
cv2.imshow('src1',  src1)

#2
src2 = cv2.imread('./data/circles2.jpg')
gray2 = cv2.cvtColor(src2,cv2.COLOR_BGR2GRAY)
circles2 = cv2.HoughCircles(gray2, method = cv2.HOUGH_GRADIENT,
          dp=1, minDist=50, param2=15, minRadius=30, maxRadius=100)

circles2 =  np.int32(circles2)
print('circles2.shape=', circles2.shape)
for circle in circles2[0,:]:    
    cx, cy, r  = circle
    cv2.circle(src2, (cx, cy), r, (0,0,255), 2) 
cv2.imshow('src2',  src2)
cv2.waitKey()
cv2.destroyAllWindows()

circles1.shape= (1, 3, 3)
circles2.shape= (1, 6, 3)


In [6]:
# inRange는 하한 값과 상한 값을 토대로 컬러 영역을 검출하는 함수이다.
import cv2
import numpy as np

#1
src1 = cv2.imread('./data/hand.jpg')
hsv1 = cv2.cvtColor(src1, cv2.COLOR_BGR2HSV)
lowerb1 = (0, 40, 0)
upperb1 = (20, 180, 255)
dst1 = cv2.inRange(hsv1, lowerb1, upperb1)

#2
src2 = cv2.imread('./data/flower.jpg')
hsv2 = cv2.cvtColor(src2,cv2.COLOR_BGR2HSV)
lowerb2 = (150, 100, 100)
upperb2 = (180, 255, 255)
dst2 = cv2.inRange(hsv2, lowerb2, upperb2)

#3
cv2.imshow('src1',  src1)
cv2.imshow('dst1',  dst1)
cv2.imshow('src2',  src2)
cv2.imshow('dst2',  dst2)
cv2.waitKey()
cv2.destroyAllWindows()


# 라벨링과 특징 파라미터

## 라벨링

영상처리에서 라벨링(Labeling)은 이미지에서 각각의 물체를 찾아 특정한 값을 부여하는 것입니다. 

라벨링은 연결된 모든 화소에 같은 번호를 붙이고 다른 연결 부분에는 다른 번호를 붙입니다. 라벨링을 할 때 고려해야 할 부분은 크게 두 가지입니다. 

첫 번째 고려해야 할 사항은 현재 주목 화소의 상하좌우 4근방 화소를 관심 화소로 할지 아니면 현재 주목 화소의 주위 8근방 화소로 할지에 따라 추출되는 물체가 달라질 수 있습니다. 

두 번째 고려해야 할 사항은 알고리즘입니다. 라벨링 알고리즘은 Grass-Fire와 2-Pass 알고리즘이 있습니다. 그리고 이 알고리즘 내에서 인접요소가 모두 라벨링 될 때까지 재귀적으로 호출하는 방식과 라벨링 정보를 스태킹하는 방식으로도 구분할 수 있습니다. 

라벨링은 다음 단계에 따라 이뤄집니다. 

1. 영상의 처리는 왼쪽에서 오른쪽으로 그리고 위에서 아래로 화소를 읽으면서 처리합니다. 가장 먼저 마주친 화소에 번호를 부여합니다. 
2. 현재 화소에 연결되고 있는 주위 화소에 라벨을 붙입니다. 
3. 2단계에서 라벨을 붙인 화소에 연결되어 있는 화소에 라벨을 붙입니다.
4. 라벨을 붙여야 할 화소가 없어질 때까지 3단계를 반복합니다.  

In [8]:
# 라벨링 함수 구현 및 적용
import numpy as np
import cv2

# 현재 주목 화소가 라벨이 붙어 있으면 연결되어 있는 주위 8근방 화소에 같은 라벨 붙이기
def labelset(img, xs, ys, label):
    height, width = img.shape
    img[ys, xs] = label
    while True:
        cnt = 0
        for y in range(1, height-1):
            for x in range(1, width-1):
                if img[y,x] == label :
                    if  img[y  , x+1] == 255 :
                        img[y  , x+1] = label; cnt = cnt+1
                    if  img[y-1, x+1] == 255 :
                        img[y-1, x+1] = label; cnt = cnt+1
                    if  img[y-1, x  ] == 255 :
                        img[y-1, x  ] = label; cnt = cnt+1
                    if  img[y-1, x-1] == 255 :
                        img[y-1, x-1] = label; cnt = cnt+1
                    if  img[y  , x-1] == 255 :
                        img[y  , x-1] = label; cnt = cnt+1
                    if  img[y+1, x-1] == 255 :
                        img[y+1, x-1] = label; cnt = cnt+1
                    if  img[y+1, x  ] == 255 :
                        img[y+1, x  ] = label; cnt = cnt+1
                    if  img[y+1, x+1] == 255 :
                        img[y+1, x+1] = label; cnt = cnt+1
        if cnt==0:
            return (0, img)
    return (1, img)

# 모든 화소에 대해 라벨링 수행하는 함수 
def labeling(img_in, base=100):
    height, width = img_in.shape
    img_label = img_in.copy()
            
    label = base # 베이스 값부터 라벨링 함
    
    for y in range(1, height-1):
        for x in range(1, width-1):
            if img_label[y,x] == 255 :
                if label >= 255:
                    print("Error! too many labels")
                    return -1
                
                _, img_label = labelset(img_label, x, y, label)
                label = label + 1
    cnt = label - base
    
    return img_label, cnt

img = cv2.imread('./data/testShapes1.jpg', 0)
ret, bin_img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV)
labeled_img, cnt = labeling(bin_img)
print("객체 수", cnt)

cv2.imshow('Labeling', labeled_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

객체 수 8


### cv2.connectedComponents(img, connectivity=8) -> cnt, labels

- img : 8비트 단일채널 이미지입니다.
- connectivity : 연결성 4 또는 8을 지정합니다. 

반환값 cnt는 라벨의 개수이며, labels는 라벨링된 출력 이미지입니다. 출력 라벨 이미지의 타입은 부호 있는 32비트입니다. 

In [9]:
import cv2
import numpy as np

img = cv2.imread('./data/testShapes1.jpg', 0)
ret, bin_img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV)
cnt, labeled_img = cv2.connectedComponents(bin_img)

print("객체 수", cnt)
print(labeled_img.dtype)

labeled_img = labeled_img*50

cv2.imshow('Labeling', labeled_img.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

객체 수 9
int32


## 특징 파라미터

특징 파라미터란 객체의 면적, 주위 길이, 원형도, 중심 위치 등을 뜻합니다. 이미지를 읽어 2진화한 후 라벨링을 수행하고 이를 기반으로 특징 파라미터들을 알 수 있습니다. 특징 파라미터 계산 절차는 다음과 같습니다. 

1. 화상 데이터 읽어들이기
2. 2진화
3. 라벨링
4. 특징 파라미터 계산

OpenCV의 함수를 이용하면 영역의 경계선을 연결한 선을 찾아 그리거나 객체의 특징 파라미터를 쉽게 얻을 수 있습니다. 

### cv2.findContours(image, mode, method) -> image, contours, hierarchy

- image : 8비트 단일채널 이미지입니다.
- mode : 경계(contour)를 찾는 방법입니다. cv2.RETR_XXX 형식입니다.
- method : 경계를 찾을 때 사용하는 근사치 방법입니다. cv2.CHAIN_XXX 형식입니다.

반환값 image, contours, hierarchy는 각각 이미지 객체와 경계들, 그리고 계층 구조를 반환합니다.

###  cv2.drawContours(image, contours, contourIdx, color, thickness) -> dst

- image : 원본 이미지입니다.
- contours : 경계 정보입니다.
- contourIdx : 경계 리스트에서 몇 번째 경계선을 그릴 것인지 지정합니다. -1이면 전체를 그립니다.
- color : 경계선 색을 지정합니다. 
- thickness : 경계선의 두께를 지정합니다. 음수이면 경계선의 내부를 채웁니다. 

반환값 dst는 이미지에 경계가 그려진 결과 이미지 객체를 반환합니다. 

In [10]:
import cv2
import numpy as np

img = cv2.imread('./data/testShapes1.jpg')
imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

ret, bin_img  = cv2.threshold(imgray,127,255,cv2.THRESH_BINARY_INV)

contours, hierachy = cv2.findContours(bin_img, 
                                      cv2.RETR_TREE,
                                      cv2.CHAIN_APPROX_SIMPLE)

image = cv2.drawContours(img, contours, -1, (0,255,0), 3)

cv2.imshow('Contours', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
import cv2
import numpy as np

img = cv2.imread('./data/lena.jpg')
imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

ret, bin_img  = cv2.threshold(imgray,127,255,0)

contours, hierachy = cv2.findContours(bin_img, 
                                      cv2.RETR_TREE,
                                      cv2.CHAIN_APPROX_SIMPLE)

image = cv2.drawContours(img, contours, -1, (0,255,0), 3)

cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
# 0706.py
import cv2
import numpy as np

#1
src = np.zeros(shape=(512,512,3), dtype=np.uint8)
cv2.rectangle(src, (50, 100), (450, 400), (255, 255, 255), -1)
cv2.rectangle(src, (100, 150), (400, 350), (0, 0, 0), -1)
cv2.rectangle(src, (200, 200), (300, 300), (255, 255, 255), -1)
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

#2
mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
##method =cv2.CHAIN_APPROX_NONE
contours, hierarchy = cv2.findContours(gray, mode, method)

print('type(contours)=', type(contours))
print('type(contours[0])=', type(contours[0]))
print('len(contours)=', len(contours))
print('contours[0].shape=', contours[0].shape)
print('contours[0]=', contours[0])

#3
cv2.drawContours(src, contours, -1, (255,0,0), 3)

#4
for pt in contours[0][:]: 
    cv2.circle(src, (pt[0][0], pt[0][1]), 5, (0,0,255), -1)

cv2.imshow('src',  src)
cv2.waitKey()
cv2.destroyAllWindows()


type(contours)= <class 'tuple'>
type(contours[0])= <class 'numpy.ndarray'>
len(contours)= 1
contours[0].shape= (4, 1, 2)
contours[0]= [[[ 50 100]]

 [[ 50 400]]

 [[450 400]]

 [[450 100]]]


In [1]:
# 0707.py
import cv2
import numpy as np

#1
src = np.zeros(shape=(512,512,3), dtype=np.uint8)
cv2.rectangle(src, (50, 100), (450, 400), (255, 255, 255), -1)
cv2.rectangle(src, (100, 150), (400, 350), (0, 0, 0), -1)
cv2.rectangle(src, (200, 200), (300, 300), (255, 255, 255), -1)
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

#2
mode = cv2.RETR_LIST
method = cv2.CHAIN_APPROX_SIMPLE;
contours, hierarchy = cv2.findContours(gray, mode, method)
##cv2.drawContours(src, contours, -1, (255,0,0), 3)  

print('len(contours)=', len(contours))
print('contours[0].shape=', contours[0].shape)
print('contours=', contours)

#3
for cnt in contours:
    cv2.drawContours(src, [cnt], 0, (255,0,0), 3)
    
    for pt in cnt: 
        cv2.circle(src, (pt[0][0], pt[0][1]), 5, (0,0,255), -1)

cv2.imshow('src',  src)
cv2.waitKey()
cv2.destroyAllWindows()


len(contours)= 3
contours[0].shape= (4, 1, 2)
contours= (array([[[200, 200]],

       [[200, 300]],

       [[300, 300]],

       [[300, 200]]], dtype=int32), array([[[ 99, 150]],

       [[100, 149]],

       [[400, 149]],

       [[401, 150]],

       [[401, 350]],

       [[400, 351]],

       [[100, 351]],

       [[ 99, 350]]], dtype=int32), array([[[ 50, 100]],

       [[ 50, 400]],

       [[450, 400]],

       [[450, 100]]], dtype=int32))
